In [10]:
#Funktion, um Benachrichtigung zu schicken

def send_mail(sender, recipient, subject, message):
    from smtplib import SMTP
    from email.mime.text import MIMEText

#header
    msg = MIMEText(message)
    sender ='Theodor.sonnenschein@web.de' 
    msg['From'] = sender
    recipient = 'Theodor.sonnenschein@web.de'
    msg['To'] = recipient
    msg['Subject'] = subject
    
#Verbindung zum Server
    server = SMTP('smtp.web.de', 587)
    server.ehlo()
    server.starttls()
    server.login(sender, 'password')
    server.sendmail(sender, recipient, msg.as_string())
    server.quit()



In [11]:
#Funktion, mit dem Messageteil

def send_alert(url, e, errtype):
    subject = "{} for {}".format(errtype, url)
    message = str(e)
    send_mail('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de', subject, message,
              )
    


In [12]:
#Link checking ohne csv-Datei
import requests

urls = [
	"https://api.github.com/invalid",
	"https://www.gutenberg.org/",
    "https://www.heise-regioconcept.de/homepages/invalid"
]
#try except Block zum Link checken
for url in urls:
    try:
        r = requests.head(url)
        r.raise_for_status()
    # Want to ensure that any HTTP error gets handled as an exception
    except requests.exceptions.HTTPError as e:
        send_alert(url, e, 'HTTPError')
    # Catch the base class exception to broadly handle all exceptions
    except requests.exceptions.RequestException as e:
        send_alert(url, e, 'Exception')

In [15]:
#E-Mail mit csv-datei
from email.message import EmailMessage
import smtplib

msg = EmailMessage()
msg["Subject"] = "Link alert"
sender ='Theodor.sonnenschein@web.de' 
msg['From'] = sender
recipient = 'Theodor.sonnenschein@web.de'
msg['To'] = recipient
#msg["From"] = "me <sender@example.org>"
#msg["To"] = "recipient <victim@example.net>"
# definitely don't mess with the .preamble

msg.set_content("Dear Team, some of the links are not working. Please have a look!")

with open("links_checked.csv", "rb") as fp:
    msg.add_attachment(
        fp.read(), maintype="application", subtype="csv")

# Notice how smtplib now includes a send_message() method
with smtplib.SMTP('smtp.web.de', 587) as s:
    s.ehlo()
    s.starttls()
    s.login(sender, 'nBbyzd2mj7DPwW7')
    s.send_message(msg)

In [13]:
#Zusammenfügen von Report erstellen und E-Mail senden:
#Funktion zum E-Mail-Senden:

def send_email(sender, recipient, subject):
    
    from email.message import EmailMessage
    import smtplib

#header

    msg = EmailMessage()
    msg["Subject"] = subject
    sender ='Theodor.sonnenschein@web.de' 
    msg['From'] = sender
    recipient = 'Theodor.sonnenschein@web.de'
    msg['To'] = recipient

#Nachricht und Anhang

    msg.set_content("Dear Team, /nsome of the links are not working. Please have a look!")

    with open("broken_links.csv", "rb") as fp:
        msg.add_attachment(
            fp.read(), maintype="broken_links", subtype="csv")
    
#Verbindung

    with smtplib.SMTP('smtp.web.de', 587) as smtplib:
        smtplib.ehlo()
        smtplib.starttls()
        smtplib.login(sender, 'nBbyzd2mj7DPwW7')
        smtplib.send_message(msg)

In [14]:
#Link checking ohne csv-Datei
import requests
import pandas as pd

urls = [
	"https://api.github.com/invalid",
	"https://www.gutenberg.org/",
    "https://www.heise-regioconcept.de/homepages/invalid"
]
#try except Block zum Link checken
broken_links = []

for url in urls:
    try:
        r = requests.head(url)
        r.raise_for_status()
        
    # All exceptions are written in a list:
    except requests.exceptions.HTTPError as http_error:
        
           broken_links.append(f"HTTP error occurred: {http_error}")
            
    except requests.exceptions.RequestException as other_error:
            broken_links.append(f"Other error occurred:' {other_error}")
            
pd.Series(list(broken_links)).to_csv("broken_links.csv")
    
    
send_alert(broken_links)

In [10]:
#Funktion, mit dem Messageteil

def send_alert(broken_links):
    subject = "Broken Links in Reading List"
    send_email('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de', subject, 
              )

In [15]:
#Link checking mit csv-Datei:

#Importieren aller notwendigen Libraries:
import csv
import pandas as pd
import requests
from email.message import EmailMessage
import smtplib

In [37]:
#Einlesen der csv-Datei mit Pandas:

reading_list = pd.read_csv("all-links_checked.csv")

In [39]:
#isolieren der einzulesenden Spalte:
links = reading_list["Link"]
links

0    https://api.github.com/invalid
1        https://www.gutenberg.org/
2        https://www.bibgndk.co.uk/
Name: Link, dtype: object

In [33]:
#request link checking:

broken_links = []

for url in links:
    try:
        r = requests.head(url)
        r.raise_for_status()
        
    # All exceptions are written in a list:
    except requests.exceptions.HTTPError as http_error:
        
           broken_links.append(f"HTTP error occurred: {http_error}")
            
    except requests.exceptions.RequestException as other_error:
            broken_links.append(f"Other error occurred:' {other_error}")
            
pd.Series(list(broken_links)).to_csv("broken_links.csv")

In [34]:
broken_links

["Other error occurred:' Invalid URL 'nan': No schema supplied. Perhaps you meant http://nan?",
 'HTTP error occurred: 404 Client Error: Not Found for url: https://api.github.com/invalid']

In [44]:
#Versuch Report mit CSV writer zu schreiben:
header = ["Link", "Error Code"]
with open("broken_links.csv", "w", newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
        

In [47]:
broken_links = []

for url in links:
    try:
        r = requests.head(url)
        r.raise_for_status()
        
    # All exceptions are written in a list:
    except requests.exceptions.HTTPError as http_error:
        
           broken_links.append(f"In {url} HTTP error occurred: {http_error}")
            
    except requests.exceptions.RequestException as other_error:
            broken_links.append(f"In {url} Other error occurred:' {other_error}")
            

Report(broken_links)
#Problem, das jedes einzelne Zeichen als eine Spalte angesehen wird.

In [46]:
def Report(broken_links):
    with open("broken_links.csv", "a", newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(broken_links)
        